In [1]:
# import packages
import pandas as pd
import math
import numpy as np
#import statsmodels.api as sm
from scipy import stats
import statistics
#import plotly.graph_objects as go
from scipy.stats import norm
#from plotly.subplots import make_subplots
#from sklearn.neighbors import KernelDensity
from scipy import optimize

In [2]:
# Load the market prices
df = pd.read_excel("/files/exercises/Homeworks/HW7/FIREDAT.xls", header=None)
df.columns = ['Date','DKK']

## Fitting a Generalized Pareto Distribution

In [3]:
u1 = 10*10**6
u2 = 20*10**6

pos1 = df['DKK']>u1
z1 = df['DKK'][pos1] - u1
pos2 = df['DKK']>u2
z2 = df['DKK'][pos2] - u2

m1 = statistics.mean(z1)
S_21 = statistics.variance(z1)
m2 = statistics.mean(z2)
S_22 = statistics.variance(z2)

ksi1 = (1 - ((m1 - u1)**2)/S_21)/2
sigma_hat1 = ((m1 - u1) / 2) * (((m1 - u1)**2) / S_21 + 1)

ksi2 = (1 - ((m2 - u2)**2)/S_22)/2
sigma_hat2 = ((m2 - u2) / 2) * (((m2 - u2)**2) / S_22 + 1)

In [34]:
def GDPLogLikelihood(param, y):
    fct = (1 + param[0] * y)**-((1 + param[0])/ param[0])/param[1]
    return np.sum(np.log(fct))

In [35]:
obs1 = z1.values
obs2 = z2.values

guesses1 = np.array([ksi1, sigma_hat1])
guesses2 = np.array([ksi2, sigma_hat2])

estimates_param1 = optimize.fmin(func = GDPLogLikelihood, x0 = guesses1, args = (obs1,))
estimates_param2 = optimize.fmin(func = GDPLogLikelihood, x0 = guesses2, args = (obs2,))

<ipython-input-34-4599922b44e5>:2: RuntimeWarning: invalid value encountered in power
  fct = (1 + param[0] * y)**-((1 + param[0])/ param[0])/param[1]
<ipython-input-34-4599922b44e5>:3: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(fct))


In [36]:
ksi1_est, sigma1_est = estimates_param1
ksi2_est, sigma2_est = estimates_param2

In [37]:
print('For u =', u1, ': ')
print()
print('MM ksi / sigma: ',ksi1,'/',sigma_hat1, '\nMLE ksi / sigma: ',ksi1_est,'/', sigma1_est)
print()
print('For u =', u2, ': ')
print()
print('MM ksi / sigma: ',ksi2,'/',sigma_hat2, '\nMLE ksi / sigma: ',ksi2_est,'/', sigma2_est)

For u = 10000000 : 

MM ksi / sigma:  0.48909448342069217 / 2071831.5157296795 
MLE ksi / sigma:  0.01834104312827539 / 3761669.0957466913

For u = 20000000 : 

MM ksi / sigma:  0.4984971665058604 / 1101949.7688230718 
MLE ksi / sigma:  0.01869364374396998 / 2000727.5490193903


In [8]:
# We will use MM parameters

p1 = statistics.mean(df['DKK'] <= u1)
p2 = statistics.mean(df['DKK'] <= u2)

sigma_tilda1 = sigma_hat1 * (1 - p1)**ksi1
sigma_tilda2 = sigma_hat2 * (1 - p2)**ksi2

mu_tilda1 = u1 - (sigma_tilda1 * ((1 - p1)**ksi1 - 1)) / ksi1
mu_tilda2 = u2 - (sigma_tilda2 * ((1 - p2)**ksi2 - 1)) / ksi2

## Computing the VaR and ES

In [9]:
p = 0.01

VaR1 = mu_tilda1 + sigma_tilda1/ksi1 * ((1 - p)**(-ksi1) - 1)
ES1 = VaR1 * (1 / (1 - ksi1) + (sigma_hat1 - ksi1 * u1) / ((1 - ksi1) * VaR1))


VaR2 = mu_tilda2 + sigma_tilda2/ksi2 * ((1 - p)**(-ksi2) - 1)
ES2 = VaR2 * (1 / (1 - ksi2) + (sigma_hat2 - ksi2 * u2) / ((1 - ksi2) * VaR2))

In [10]:
print('1st VaR: ',VaR1, '\n2nd VaR: ',VaR2)

1st VaR:  10846054.418208241 
2nd VaR:  20297748.27793524


In [18]:
# With normal assumption and kernel estimation
losses = df['DKK']
mu = statistics.mean(losses)
sigma = statistics.variance(losses)

In [19]:
# Not sure if we have to take all the data set or just the one above the threshold

VaR_gaussian0 = - mu - math.sqrt(sigma)*stats.norm.ppf(0.01)
VaR_gaussian1 = - m1 - math.sqrt(S_21)*stats.norm.ppf(0.01)
VaR_gaussian2 = - m2 - math.sqrt(S_22)*stats.norm.ppf(0.01)

In [28]:
print('0th VaR: ',VaR_gaussian0, '(We like this one)','\n1st VaR: ',VaR_gaussian1, '\n2nd VaR: ',VaR_gaussian2)

0th VaR:  17298310.12222603 (We like this one) 
1st VaR:  49822648.908226356 
2nd VaR:  71040637.23970503


In [14]:
def gaussianVaR(VaR, obs, bandwidth, alpha):
    return (np.mean(stats.norm.cdf((-VaR - obs)/bandwidth))-alpha)**2

In [23]:
obs = np.array(losses)
bandwidth = pow(len(losses), -0.2) * pow(statistics.variance(losses), 0.5)
alpha = 0.01
guesses3 = VaR_gaussian1
guesses4 = VaR_gaussian2
guesses5 = VaR_gaussian0

In [24]:
estimates_Var0 = optimize.fmin(func = gaussianVaR, x0 = guesses5, args = (obs, bandwidth, alpha,))
estimates_Var1 = optimize.fmin(func = gaussianVaR, x0 = guesses3, args = (obs, bandwidth, alpha,))
estimates_Var2 = optimize.fmin(func = gaussianVaR, x0 = guesses4, args = (obs, bandwidth, alpha,))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 42
         Function evaluations: 84
Optimization terminated successfully.
         Current function value: 0.000100
         Iterations: 36
         Function evaluations: 107
Optimization terminated successfully.
         Current function value: 0.000100
         Iterations: 37
         Function evaluations: 110


In [29]:
print('0th VaR: ',estimates_Var0[0], '(We like this one)','\n1st VaR: ',estimates_Var1[0], '\n2nd VaR: ',estimates_Var2[0])

0th VaR:  2950677.710089679 (We like this one) 
1st VaR:  49822648.908226356 
2nd VaR:  71040637.23970503
